Smurfette principle derivation, analysis of movie summaries, women are more often than not mentioned with a male character.

Process the summary, and count how many times each character is mentioned, use n-grams (2-3 ?) to see when they are mentioned together.

Maybe measure throughout the years ?

Or measure with different n for n-grams, interactive plot ? (possible to do both yearly + variable n too)

In [130]:
import pandas as pd
import numpy as np
import json
import string
import os
import csv
import spacy
from spacy.lang.en import English

#Punctuations from python string.punctuation without the dot ('.'), additional punctuation added to remove artifacts
punctuation = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '/', ':',
 ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'] + ['``', '\'\'', '\'s']

In [131]:
# Character metadata
char_md_cols = ['Wikipedia movie ID',
'Freebase movie ID',
'Movie release date',
'Character name',
'Actor date of birth',
'Actor gender',
'Actor height (in meters)',
'Actor ethnicity (Freebase ID)',
'Actor name',
'Actor age at movie release',
'Freebase character/actor map ID',
'Freebase character ID',
'Freebase actor ID',
]
char_md = pd.read_csv('data/character.metadata.tsv', sep='\t', names=char_md_cols)
print('Number of characters:', char_md.shape[0])
char_md.head(2)

Number of characters: 450669


,Wikipedia movie ID,Freebase movie ID,Movie release date,Character name,Actor date of birth,Actor gender,Actor height (in meters),Actor ethnicity (Freebase ID),Actor name,Actor age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.62,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.78,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4


In [132]:
char_to_gender = char_md[['Wikipedia movie ID', 'Character name', 'Actor gender']].dropna().drop_duplicates()

In [133]:
wikipedia_movie_IDS = os.listdir("./data/corenlp_plot_summaries/")
wikipedia_movie_IDS = [filename.rsplit('.', 2)[0] for filename in wikipedia_movie_IDS]

In [134]:
len(wikipedia_movie_IDS)

42306

In [135]:
import gzip
from xml.dom.minidom import parseString #, parse 


def open_parse_summary(wiki_id):
    # open and read gzipped xml file
    xml_file = gzip.open("./data/corenlp_plot_summaries/" + wiki_id + ".xml.gz")
    document = parseString(xml_file.read())
    
    #Read all words
    words = document.getElementsByTagName('word')
    #lemmas = dom.getElementsByTagName('lemma')
    
    #Looks for the desired node type and extracts their values
    words_list = [word.childNodes[0].nodeValue for word in words]
    #lemmas_list = [lemma.childNodes[0].nodeValue for lemma in lemmas]
    
    return words_list#, lemmas_list

In [219]:
# DO NOT RUN UNLESS NECESSARY !!!!!
i = 0
tot = len(wikipedia_movie_IDS)

#for wiki_id in wikipedia_movie_IDS[0:20]:
    print(wiki_id)
    summary = list(open_parse_summary(wiki_id))
    with open("./data/corenlp_summaries_words/"+ wiki_id + ".txt", 'w') as file_words:
        for word in summary:
            if word not in punctuation:
                # write each item on a new line
                file_words.write("%s\n" % word)
    i += 1
    print(str(i) + "/" + str(tot))

26167604
1/42306
14075290
2/42306
24379585
3/42306
8683288
4/42306
18040146
5/42306
3113630
6/42306
15117612
7/42306
9404578
8/42306
21411112
9/42306
23553986
10/42306
9149984
11/42306
327582
12/42306
25375805
13/42306
5144523
14/42306
2817162
15/42306
24778429
16/42306
4298666
17/42306
8318237
18/42306
34383775
19/42306
15025803
20/42306


In [136]:
wikipedia_movie_IDS[0:5]

['26167604', '14075290', '24379585', '8683288', '18040146']

In [137]:
char_to_gender

,Wikipedia movie ID,Character name,Actor gender
0,975900,Akooshay,F
1,975900,Lieutenant Melanie Ballard,F
2,975900,Desolation Williams,M
3,975900,Sgt Jericho Butler,M
4,975900,Bashira Kincaid,F
...,...,...,...
450658,913762,Lord Feff,M
450659,913762,Additional Voices,F
450661,913762,UN Spacy Commander,M
450662,913762,Silvie Gena,F


In [138]:
empty_count = 0
avg_len = 0

for wiki_id in wikipedia_movie_IDS:
    test = char_to_gender[char_to_gender['Wikipedia movie ID'] == int(wiki_id)]
    if(test.empty):
        empty_count += 1
    else:
        avg_len += len(test)
        
avg_len /= (len(wikipedia_movie_IDS)-empty_count)

print("Average of " + str(avg_len) + " actors per movie")

KeyboardInterrupt: 

In [139]:
spacy_stopwords_and_general = spacy.lang.en.stop_words.STOP_WORDS
#TODO: Add eventual additionnal words
# Additional elements: man/wm/hb/wife cannot be processed as characters without more thorough language processing
# Numbers: characters are sometimes described as for example 'Bartender 1', 'Bartender 2', these need to be avoided
additional_elems = ['man', 'woman', 'husband', 'wife'] + ['0','1', '2', '3', '4', '5', '6', '7', '8', '9']
spacy_stopwords_and_general.update(additional_elems)

def make_MF_repr(wiki_id):
    with open("./data/corenlp_summaries_words/"+ wiki_id + ".txt", 'r') as summary_file:
        #Read the summary
        summary_text = summary_file.read().replace('\n', ' ')
        
        #Get the characters from this movie
        chars = char_to_gender[char_to_gender['Wikipedia movie ID'] == int(wiki_id)][['Character name', 'Actor gender']]
        chars = dict(zip(chars['Character name'], chars['Actor gender']))
        
        #Remove punctuation from character names
        for char_name in list(chars):
            char_name_new = char_name
            for pun in punctuation:
                char_name_new = char_name_new.replace(pun, ' ')
            chars[char_name_new] = chars.pop(char_name)
        
        #Remove characters which contain a stopword to keep (mostly) relevant characters (names).
        for char_name in list(chars):
            for word in char_name.split(' '):
                if word.lower() in spacy_stopwords_and_general:
                    chars.pop(char_name)
                    break
                    
        #Replace character's names with M or F respectively
        for char_name in list(chars):
            summary_text = summary_text.replace(char_name, chars[char_name])
            for word in char_name.split(' '):
                summary_text = summary_text.replace(word, chars[char_name])
                
        # Save as final file format (M and F stay, other words get replaced by the number of words between two M/F)
        # Each line is a new sentence
        # For example, if the only character is named 'Bob' (man), 'Bob is cool be like Bob. I like Bob.' becomes:
        # M4M
        # 2M
        #Check if chars list not empty
        if(bool(chars)):
            with open("./data/corenlp_summaries_encoded/"+ wiki_id + ".txt", 'w') as file_encoded:
                count_non_character = 0
                for word in summary_text.split(' '):
                    if word == 'M' or word == 'F':
                        if count_non_character != 0:
                            file_encoded.write("%s" % str(count_non_character))
                        file_encoded.write("%s" % word)
                        count_non_character = 0
                    elif word == '.':
                        if count_non_character != 0:
                            file_encoded.write("%s" % str(count_non_character))
                        file_encoded.write("\n")
                        count_non_character = 0
                    else:
                        count_non_character += 1

In [149]:
# Do not rerun unless necessary
#for wiki_id in wikipedia_movie_IDS[0:20]:
    make_MF_repr(wiki_id)

IndentationError: unexpected indent (3493001002.py, line 3)

In [141]:
# Counts the total occurence of M/F characters in the summary.
def count_pure_M_F(wiki_id):
    with open("./data/corenlp_summaries_encoded/"+ wiki_id + ".txt", 'r') as summary_file:
        summary_str = summary_file.read()
        return summary_str.count('M'), summary_str.count('F')

In [142]:
# Counts the occurence of M and F at a distance <= n. For ex. 'M2F' would give 1 for n=3 but 0 for n=2 or n=1
# Not specifying n means taking the whole sentence
def count_co_occ_M_F(wiki_id, n = 1000):
    with open("./data/corenlp_summaries_encoded/"+ wiki_id + ".txt", 'r') as summary_file:
        summary_str = summary_file.read()
        co_occ_sum = {'M': 0, 'F': 0}
        for line in summary_str.split('\n'):
            previous = ''
            indic_hasbeencounted = 0
            int_saver_M_F = {'M':0, 'F':0}
            strint_saver = '0'
            for word in line:
                if word == 'M' or word == 'F':
                    opposite_word = 'F' if word == 'M' else 'F'
                    int_saver_M_F[opposite_word] += int(strint_saver)
                    if word != previous and previous and int_saver_M_F[opposite_word] <= n-1:
                        co_occ_sum[word] += 1
                        int_saver_M_F[word] = 0
                        if not indic_hasbeencounted:
                            co_occ_sum[previous] += 1
                        indic_hasbeencounted = 1
                    else:
                        indic_hasbeencounted = 0
                    previous = word
                    strint_saver = '0'
                else:
                    strint_saver += word
        return co_occ_sum['M'], co_occ_sum['F']

In [146]:
# Counts the occurence of M and F at a distance <= n. For ex. 'M2F' would give 1 for n=3 but 0 for n=2 or n=1
# Not specifying n means taking the whole sentence
def count_co_occ_M_F(wiki_id, n = 1000):
    with open("./data/corenlp_summaries_encoded/"+ wiki_id + ".txt", 'r') as summary_file:
        summary_str = summary_file.read()
        co_occ_sum = {'M': 0, 'F': 0}
        for line in summary_str.split('\n'):
            distances = {'M': np.array([]), 'F': np.array([])}
            indicators_counted = {'M': np.array([]), 'F': np.array([])}
            strint_saver = '0'
            for word in line:
                if word == 'M' or word == 'F':
                    
                    # Gets the opposite word
                    opposite_word = 'F' if word == 'M' else 'M'
                    
                    # Adds the current word in the distances and indicators
                    distances[word] += int(strint_saver) + 1
                    distances[word] = np.append(distances[word], 0)
                    if not distances[opposite_word].size == 0:
                        distances[opposite_word] += int(strint_saver) + 1
                        
                        # Gets the indices where the distance is < n and the word hasn't been counted in the sum
                        indices_dist = np.where(distances[opposite_word] <= n)[0]
                        indices_indic = np.where(indicators_counted[opposite_word] == 0)[0]
                        valid_indices = np.intersect1d(indices_dist, indices_indic)
                        
                        # If there was an occurence of the opposite gender at a valid distance
                        if indices_dist.size >= 1:
                            
                            # Increases the sum of the current word by 1 and indicates that occurence as counted
                            co_occ_sum[word] += 1
                            indicators_counted[word] = np.append(indicators_counted[word], 1)
                            if indices_indic.size >= 1:
                                
                                # Increases the sum of the opposite word by the number of previous uncounted occurences
                                # and indicates these occurences as counted
                                co_occ_sum[opposite_word] += valid_indices.size
                                indicators_counted[opposite_word][valid_indices] = 1
                        else:
                            
                            # Indicate the current occurence as uncounted
                            indicators_counted[word] = np.append(indicators_counted[word], 0)
                    else:
                        indicators_counted[word] = np.append(indicators_counted[word], 0)
                        
                    # Reset the strint_saver
                    strint_saver = '0'
                else:
                    
                    # Appends the strint saver with the current number
                    strint_saver += word
        return co_occ_sum['M'], co_occ_sum['F']

### Creating the final counts for graphing

Given (start_n, end_n), computes the total M/F representations in the summaries.

N represents the maximal distance between M and F to be counted as a co-occurence

Saves the results in movie_MF_data.csv in the following form :

[wiki_id_1, counts_1_start_n, ..., counts_1_end_n]  
[...]  
[wiki_id_final, counts_final_start_n, ..., counts_final_end_n]  
[mean_counts]

In [147]:
#Makes the header for the csv
def make_csv_header(start_n, end_n):
    csv_header = ['wiki_id']
    for n in range(start_n, end_n+1):
        csv_header.extend(['total_M_'+str(n), 'total_F_'+str(n), 'c_o_M_'+str(n), 'c_o_F_'+str(n),
                            'pure_M_'+str(n), 'pure_F_'+str(n),])
    csv_header.extend(['total_M_sent', 'total_F_sent', 'c_o_M_sent', 'c_o_F_sent',
                            'pure_M_sent', 'pure_F_sent'])
    return csv_header

def make_csv_linepart(wiki_id, n):
    temp_line = []
    
    #Computes the total M, F
    total_M, total_F = count_pure_M_F(wiki_id)
    temp_line.extend([total_M, total_F])
    
    #Computes the co_occurences with given distance
    co_occ_M, co_occ_F = count_co_occ_M_F(wiki_id, n)
    temp_line.extend([co_occ_M, co_occ_F])
    
    #Computes the pure M, F
    pure_M = total_M - co_occ_M
    pure_F = total_F - co_occ_F
    temp_line.extend([pure_M, pure_F])
    
    return temp_line

#Makes the final counts for each movie
def make_final_file(start_n, end_n):
    csv_header = make_csv_header(start_n, end_n)
    files = [filename.split('.txt')[0] for filename in os.listdir('./data/corenlp_summaries_encoded/')]
    with open('./data/movie_MF_data_'+str(start_n)+'_'+str(end_n)+'.csv', 'w', encoding='UTF8', newline='') as final_file:
        #Write the header
        writer = csv.writer(final_file)
        writer.writerow(csv_header)
        
        #Saving values for mean (6 per n and 6 for whole sentences)
        mean_counts = np.zeros((end_n-start_n+2)*6 )
        
        #Write the counts for each file, for each possible n
        for wiki_id in files:
            final_file_line = [wiki_id]
            
            #Computes the counts for each n
            for n in range(start_n, end_n+1):
                final_file_line.extend(make_csv_linepart(wiki_id, n))
                
            #Computes the counts for whole sentences.
            final_file_line.extend(make_csv_linepart(wiki_id, 1000))
            mean_counts += np.array(final_file_line[1:])
            writer.writerow(final_file_line)
        mean_counts /= len(files)
        means_line = ['means']
        means_line.extend(mean_counts.tolist())
        writer.writerow(means_line)

In [150]:
make_final_file(2, 10)